<a href="https://colab.research.google.com/github/Kimtaewookdev/Sec1_project/blob/main/AI_13_%EA%B9%80%ED%83%9C%EC%9A%B1_Section1_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 다음 분기 게임 설계


In [1]:
import pandas as pd
import numpy as np

### 데이터 전처리

In [2]:
data_url = 'https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv'
data = pd.read_csv(data_url, index_col=0)

df = pd.DataFrame(data)
df.info()
# 'Year', 'Genre', 'Publisher'에 결측치 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16598 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.3+ MB


In [3]:
# 연도 특성의 00 년 단위 데이터 조정
for i in range(1, len(df)+1):
  if df.at[i,'Year'] <= 50:
    df.at[i,'Year'] += 2000
  elif 50 < df.at[i,'Year'] <= 100:
    df.at[i,'Year'] += 1900

In [4]:
# 퍼블리셔 특성의 'Unknown'을 결측치로 취급
df['Publisher'] = df['Publisher'].replace('Unknown', np.NAN)

In [5]:
# 출고량 단위는 표기되어있지않으나 M(백만 장)으로 추측
# 출고량 데이터 중 단위 M이 남아있거나 K(천 장)이 표기된 것 있음
# M은 제거하고 K는 0.001을 곱해 M으로 환산

for col in df.columns[-4:]:
  df[col] = df[col].replace({"K":"*1e-3", "M":""}, regex=True).apply(pd.eval)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16598 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16337 non-null  object 
 5   NA_Sales     16598 non-null  float64
 6   EU_Sales     16598 non-null  float64
 7   JP_Sales     16598 non-null  float64
 8   Other_Sales  16598 non-null  float64
dtypes: float64(5), object(4)
memory usage: 1.8+ MB


In [6]:
# 전세계 출고량 특성 추가
df['Total_Sales'] = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales']

###  지역에 따라서 선호하는 게임 장르 분석

- 지역의 장르별 출고량 비중
- 지역의 장르별 평균 출고량 비중

In [35]:
## 장르 결측치 제거

df_g = df.dropna(subset = ['Genre'])
sales = df.columns[-5:-1]
col_g = sales.insert(0, 'Genre')

def get_top(df, n=3):
  top = dict()
  for col in df.columns:
    top[col] = dict(df[col].sort_values(ascending=False).head(n)).items()
  return pd.DataFrame(top).T

# 각 지역의 장르별 출고량 상위
df_g_sum = df_g[col_g].groupby('Genre').sum().round(3)
get_top(df_g_sum)

,0,1,2
NA_Sales,"(Action, 874.81)","(Sports, 680.52)","(Shooter, 581.92)"
EU_Sales,"(Action, 522.16)","(Sports, 376.48)","(Shooter, 313.05)"
JP_Sales,"(Role-Playing, 344.95)","(Action, 157.9)","(Sports, 135.37)"
Other_Sales,"(Action, 186.45)","(Sports, 134.67)","(Shooter, 102.65)"


In [8]:
#각 지역의 장르별 평균 출고량 상위
df_g_mean = df_g[col_g].groupby('Genre').mean().round(3)
get_top(df_g_mean)

,0,1,2
NA_Sales,"(Platform, 0.505)","(Shooter, 0.445)","(Sports, 0.291)"
EU_Sales,"(Shooter, 0.239)","(Platform, 0.228)","(Racing, 0.191)"
JP_Sales,"(Role-Playing, 0.233)","(Platform, 0.148)","(Fighting, 0.103)"
Other_Sales,"(Shooter, 0.078)","(Racing, 0.062)","(Platform, 0.058)"


### 연도별 게임의 트렌드 분석

- 연도마다 게임 수가 많은 장르 
- 연도마다 Total sales 합계가 가장 높은 장르
- 연도마다 Total sales 평균이 가장 높은 장르

1997년부터 2016년까지의 데이터를 고려, 그외 연도는 표본 개수가 현저히 적기에 제외

In [36]:
# 연도, 장르 결측치 제거
df_yg = df.dropna(subset = ['Year', 'Genre'])

# 연도별 게임 발매수가 많은 장르
genre_n_py = pd.crosstab(df_yg['Year'], df_yg['Genre'])[1997:2016]
get_top(genre_n_py.T)

,0,1,2
1997.0,"(Sports, 46)","(Racing, 36)","(Action, 31)"
1998.0,"(Sports, 70)","(Racing, 48)","(Action, 44)"
1999.0,"(Sports, 62)","(Racing, 49)","(Action, 39)"
2000.0,"(Sports, 81)","(Action, 43)","(Racing, 43)"
2001.0,"(Sports, 96)","(Racing, 71)","(Action, 66)"
2002.0,"(Sports, 188)","(Action, 125)","(Racing, 103)"
2003.0,"(Action, 144)","(Sports, 125)","(Racing, 109)"
2004.0,"(Action, 125)","(Sports, 103)","(Shooter, 88)"
2005.0,"(Action, 192)","(Sports, 122)","(Misc, 115)"
2006.0,"(Action, 184)","(Sports, 138)","(Misc, 109)"


In [30]:
# 연도별 전세계 출고량 합계가 가장 높은 장르
genre_sum_py = pd.pivot_table(df_yg,
                              values='Total_Sales',
                              index=['Year'],
                              columns=['Genre'],
                              aggfunc=np.sum, 
                              fill_value=0)[1997:2016].round(3)

get_top(genre_sum_py.T)

,0,1,2
1997.0,"(Racing, 31.92)","(Sports, 30.04)","(Action, 27.59)"
1998.0,"(Sports, 41.75)","(Action, 39.41)","(Fighting, 31.68)"
1999.0,"(Role-Playing, 47.24)","(Racing, 37.19)","(Sports, 30.21)"
2000.0,"(Sports, 41.24)","(Action, 33.89)","(Role-Playing, 29.04)"
2001.0,"(Action, 56.68)","(Racing, 55.83)","(Sports, 51.47)"
2002.0,"(Action, 86.65)","(Sports, 65.31)","(Shooter, 47.64)"
2003.0,"(Action, 67.98)","(Sports, 55.96)","(Racing, 52.18)"
2004.0,"(Action, 76.15)","(Sports, 61.11)","(Role-Playing, 53.95)"
2005.0,"(Action, 85.52)","(Misc, 61.13)","(Sports, 59.53)"
2006.0,"(Sports, 136.12)","(Misc, 67.23)","(Action, 66.49)"


In [11]:
# 연도별 게임의 전세계 출고량 평균이 가장 높은 장르
genre_mean_py = pd.pivot_table(df_yg,
                               values='Total_Sales',
                               index=['Year'],
                               columns=['Genre'],
                               aggfunc=np.mean,
                               fill_value=0)[1997:2016].round(3)

get_top(genre_mean_py.T)

,0,1,2
1997.0,"(Platform, 1.214)","(Role-Playing, 0.949)","(Action, 0.89)"
1998.0,"(Platform, 1.193)","(Role-Playing, 1.08)","(Action, 0.896)"
1999.0,"(Role-Playing, 1.476)","(Platform, 1.156)","(Misc, 0.827)"
2000.0,"(Role-Playing, 1.001)","(Action, 0.788)","(Misc, 0.778)"
2001.0,"(Fighting, 1.006)","(Platform, 0.915)","(Action, 0.859)"
2002.0,"(Role-Playing, 0.835)","(Action, 0.693)","(Shooter, 0.611)"
2003.0,"(Role-Playing, 0.631)","(Simulation, 0.606)","(Fighting, 0.541)"
2004.0,"(Role-Playing, 0.899)","(Platform, 0.717)","(Racing, 0.652)"
2005.0,"(Simulation, 1.297)","(Racing, 0.732)","(Puzzle, 0.633)"
2006.0,"(Sports, 0.986)","(Platform, 0.92)","(Misc, 0.617)"


### 출고량 높은 게임에 대한 분석 및 시각화 프로세스

다음 분기 게임 설계

지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.

연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.

출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함되어야 합니다.

개인적인 분석 목표, 질문

- 플랫폼 장르에는 마리오 시리즈, 롤플레잉에는 포켓몬 시리즈같은 대형 ip가 있어 크게 잡히는 것일 수 있다. 따라서 장르와 퍼블리셔와 플랫폼에 대한 분석이 필요하다.